# Evaluating your trained AWS DeepRacer models

This notebook describes how to run evaluation independent of the training job during or after the completition of the training job. We recommend that you wait until the model training graphs indicate the models are converging before running this notebook.

* It may be prudent to run evaluation of your models using a different track to determine whether the agent is overfitting to the current track or environment settings. But ensure that the final submission uses the <<<>>> track


## Prequisites

Obtain the following from your training job. It will be printed out when you run the training notebook. Copy the corresponding fields. 

 - S3 bucket: where your models are saved
 - S3_prefix: your job prefix
 - WORLD_NAME: indicates the track to run evaluation on
 - simulation_app_arn: simulation job resource locator
 
The simulation application identification is needed to locate your AWS RoboMaker simulation job with the corresponding Gazebo assets and reinforcement learning files.

In [ ]:
WORLD_NAME = '<CHOOSE YOUR TRACK NAME>'
s3_bucket = '<FILL IN YOUR S3 Bucket>'
s3_prefix = '<FILL IN YOUR S3 Prefix>'
# the simulation arn looks like this "arn:aws:robomaker:us-west-2:xxx:simulation-application/xxx/xxx"
simulation_app_arn = '<FILL IN YOUR S3 Prefix>'
                   

## Imports

The following libraries are needed throughout the notebook. We collected them all here. 

** If your notebook can not import use "!pip install ..." to install the corresponding library.

In [ ]:
import boto3
import sagemaker
import sys
import os
import re
import numpy as np
import subprocess
sys.path.append("common")
from misc import get_execution_role, wait_for_s3_object
from docker_utils import build_and_push_docker_image
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework
from time import gmtime, strftime
import time
from IPython.display import Markdown
from markdown_helper import *
from copy_to_sagemaker_container import get_sagemaker_docker, copy_to_sagemaker_container, get_custom_image_name

## Select instance type and duration

Create a unique job name. The default duration for the evaluation is 2 hours. Adjust as needed. 

In [ ]:
# Select the instance type
instance_type = "ml.p3.8xlarge"
#instance_type = "ml.c5.4xlarge"

# Starting SageMaker session
sage_session = sagemaker.session.Session()

# Create unique job name.
job_name_prefix = '<MY JOB NAME PREFIX>'

# Duration of job in seconds (1 hours), usually it should take only a few mins
job_duration_in_seconds = int(60 * 60 * 2)

# AWS Region
aws_region = sage_session.boto_region_name
if aws_region not in ["us-west-2", "us-east-1", "eu-west-1"]:
    raise Exception("This notebook uses RoboMaker which is available only in US East (N. Virginia),"
                    "US West (Oregon) and EU (Ireland). Please switch to one of these regions.")

In [ ]:
try:
    sagemaker_role = sagemaker.get_execution_role()
except:
    sagemaker_role = get_execution_role('sagemaker')

print("Using Sagemaker IAM role arn: \n{}".format(sagemaker_role))

In [ ]:
ec2 = boto3.client('ec2')

#
# Check if the user has Deepracer-VPC and use that if its present. This will have all permission.
# This VPC will be created when you have used the Deepracer console and created one model atleast
# If this is not present. Use the default VPC connnection
#
deepracer_security_groups = [group["GroupId"] for group in ec2.describe_security_groups()['SecurityGroups']\
                             if group['GroupName'].startswith("deepracer-vpc")]
if(deepracer_security_groups):
    print("Using the DeepRacer VPC stacks")
    deepracer_vpc = [vpc['VpcId'] for vpc in ec2.describe_vpcs()['Vpcs'] \
                     if "Tags" in vpc for val in vpc['Tags'] \
                     if val['Value'] == 'deepracer-vpc'][0]
    deepracer_subnets = [subnet["SubnetId"] for subnet in ec2.describe_subnets()["Subnets"] \
                         if subnet["VpcId"] == deepracer_vpc]
else:
    print("Using the default VPC stacks")
    deepracer_vpc = [vpc['VpcId'] for vpc in ec2.describe_vpcs()['Vpcs'] if vpc["IsDefault"] == True][0]

    deepracer_security_groups = [group["GroupId"] for group in ec2.describe_security_groups()['SecurityGroups'] \
                                 if 'VpcId' in group and group["GroupName"] == "default" and group["VpcId"] == deepracer_vpc]

    deepracer_subnets = [subnet["SubnetId"] for subnet in ec2.describe_subnets()["Subnets"] \
                         if subnet["VpcId"] == deepracer_vpc and subnet['DefaultForAz']==True]

print("Using VPC:", deepracer_vpc)
print("Using security group:", deepracer_security_groups)
print("Using subnets:", deepracer_subnets)

In [ ]:
#TODO: Explain to customer what CREATE_ROUTE_TABLE is doing
CREATE_ROUTE_TABLE = True

def create_vpc_endpoint_table():
    print("Creating ")
    try:
        route_tables = [route_table["RouteTableId"] for route_table in ec2.describe_route_tables()['RouteTables']\
                        if route_table['VpcId'] == deepracer_vpc]
    except Exception as e:
        if "UnauthorizedOperation" in str(e):
            display(Markdown(generate_help_for_s3_endpoint_permissions(sagemaker_role)))
        else:
            display(Markdown(create_s3_endpoint_manually(aws_region, deepracer_vpc)))
        raise e

    print("Trying to attach S3 endpoints to the following route tables:", route_tables)

    assert len(route_tables) >= 1, "No route tables were found. Please follow the VPC S3 endpoint creation "\
                                  "guide by clicking the above link."

    try:
        ec2.create_vpc_endpoint(DryRun=False,
                                VpcEndpointType="Gateway",
                                VpcId=deepracer_vpc,
                                ServiceName="com.amazonaws.{}.s3".format(aws_region),
                                RouteTableIds=route_tables)
        print("S3 endpoint created successfully!")
    except Exception as e:
        if "RouteAlreadyExists" in str(e):
            print("S3 endpoint already exists.")
        elif "UnauthorizedOperation" in str(e):
            display(Markdown(generate_help_for_s3_endpoint_permissions(role)))
            raise e
        else:
            display(Markdown(create_s3_endpoint_manually(aws_region, default_vpc)))
            raise e

if CREATE_ROUTE_TABLE:
    create_vpc_endpoint_table()

## Evaluation - Track

The rest will run the evaluation...

In [ ]:
# SDK appends the job name and output folder
s3_output_path = 's3://{}/'.format(s3_bucket)

# Get the AWS account id of this account
sts = boto3.client("sts")
account_id = sts.get_caller_identity()['Account']

print("Using s3 bucket {}".format(s3_bucket))
print("Model checkpoints and other metadata will be stored at: \ns3://{}/{}".format(s3_bucket, s3_prefix))

In [ ]:
robomaker = boto3.client("robomaker")

### Model Evaluation

We require you to evaluate your model on track "<<<>>>" for the submission

In [ ]:
sys.path.append("./src")

num_simulation_workers = 1

envriron_vars = {
    "WORLD_NAME": WORLD_NAME,  ### Make sure you evaluate on track named -- "???" ####
    "KINESIS_VIDEO_STREAM_NAME": "SilverstoneStream",
    "SAGEMAKER_SHARED_S3_BUCKET": s3_bucket,
    "SAGEMAKER_SHARED_S3_PREFIX": s3_prefix,
    "MODEL_S3_BUCKET": s3_bucket,
    "PYTHONPATHS":"wohooo this worked",
    "MODEL_S3_PREFIX": s3_prefix,
    "ALTERNATE_DRIVING_DIRECTION": "false",
    "APP_REGION": aws_region,
    "MODEL_METADATA_FILE_S3_KEY": "%s/model_metadata.json" % s3_prefix,
    "METRICS_S3_BUCKET": s3_bucket,
    "METRICS_S3_OBJECT_KEY": s3_prefix + "/evaluation_metrics.json",
    "NUMBER_OF_TRIALS": "15", # Default
    "ROBOMAKER_SIMULATION_JOB_ACCOUNT_ID": account_id
}


simulation_application = {
    "application":simulation_app_arn,
    "launchConfig": {
         "packageName": "deepracer_simulation_environment",
         "launchFile": "evaluation.launch",
         "environmentVariables": envriron_vars
    }
}
                            
vpcConfig = {"subnets": deepracer_subnets,
             "securityGroups": deepracer_security_groups,
             "assignPublicIp": True}

responses = []
for job_no in range(num_simulation_workers):
    response =  robomaker.create_simulation_job(clientRequestToken=strftime("%Y-%m-%d-%H-%M-%S", gmtime()),
                                                outputLocation={ 
                                                  "s3Bucket": s3_bucket,
                                                  "s3Prefix": s3_prefix
                                                },
                                                maxJobDurationInSeconds=job_duration_in_seconds,
                                                iamRole=sagemaker_role,
                                                failureBehavior="Continue",
                                                simulationApplications=[simulation_application],
                                                vpcConfig=vpcConfig)
    responses.append(response)

print("Created the following EVALUATION jobs:")
job_arns = [response["arn"] for response in responses]
for response in responses:
    print("Evaluation Job ARN", response["arn"]) 

In [ ]:
# Evaluation link
display(Markdown(generate_robomaker_links(job_arns, aws_region)))

## Copy the output of the following cells for the submission notebook

In [ ]:
arn_str = list()
for response in responses:
    arn_str.append(response["arn"].split('/')[-1])
print('stream_name_list =' + str(arn_str))
print("s3_bucket = '%s'" % s3_bucket)
print("s3_prefix = '%s'" % s3_prefix)

## Lap Times from the evaluation run

Go to notebook 1_logs_evaluation-NeurIPS.ipynb to plot the lap times and debug your model evaluation